In [ ]:
import os
import numpy as np
import ants
import mcubes
import trimesh
from datasets.preprocess import process_scan
from datasets.GI_generator_support import process_point_cloud_single_patient
import tqdm

In [ ]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
# data preprocessing

BASE_DIR = 'skull_stripping'
template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
template_img_ants = ants.image_read(template_img_path, reorient='LPI')

mask_template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a_mask.nii')
mask_template_img_ants = ants.image_read(mask_template_img_path, reorient='LPI')

data_pth = 'mytest/'
tumor_path_all =data_pth+'tumor_data'
tumor_patients = np.load(data_pth+'data_utils/tumor_patients.npy')
patients=np.load(data_pth+'data_utils/H_patients.npy')


images_path = data_pth+'H_data' + '/' + 'Original'
labels_path = data_pth+ 'H_data' + '/' + 'STAPLE'
center_359 = np.load('data_utils/center359.npy')
center_tumor = np.load('data_utils/centerT125.npy')
center_T = center_tumor
center_H = center_359
center_T = [center_T[i] for i in range(len(center_T))]
center_H = [center_H[i] for i in range(len(center_H))]
skull_brain_path = []
brain_masks = []

for p_fold in listdir_nohidden(images_path):
    patient_id = p_fold[:-7]
    if len(patient_id)>2 and 'nii.gz' in p_fold:
        BrainSkull_path = images_path + '/' + patient_id + '.nii.gz'
        Brain_mask = labels_path + '/' + patient_id + '_staple.nii.gz'
        skull_brain_path.append(BrainSkull_path) 
        brain_masks.append(Brain_mask) 

                
skull_brain_path_order = []
for j in range(len(patients)):
    for i in range(len(skull_brain_path)):

        if patients[j] in skull_brain_path[i].split('/')[-1].split('.')[0]:
            skull_brain_path_order.append(skull_brain_path[i])
brain_masks_order = []
for j in range(len(patients)):
    for i in range(len(brain_masks)):

        if  patients[j] in brain_masks[i].split('/')[-1].split('.')[0]:   
     
            brain_masks_order.append(brain_masks[i])     
            
skull_brain_path = skull_brain_path_order
brain_masks = brain_masks_order

tumor_skull_path = []
tumor_brain_masks = []

for data_folder in listdir_nohidden(tumor_path_all):
    folder_path = tumor_path_all + '/' + data_folder
    
    files =[]
    for p_fold in listdir_nohidden(folder_path):
        files.append(p_fold)    
    tumor_skull_path.append(folder_path + '/' + 'T1C.nii.gz')
    if 'T1C_brain_mask_mc.nii.gz' in files:
        tumor_brain_masks.append(folder_path + '/' + 'T1C_brain_mask_mc.nii.gz')
    elif 'T1C_brain_mask_m.nii.gz' in files:
        tumor_brain_masks.append(folder_path + '/' + 'T1C_brain_mask_m.nii.gz')
    else:
        print('error  ',folder_path)
    
tumor_skull_path_order = []
tumor_brain_masks_order = []

for j in range(len(tumor_patients)):
    for i in range(len(tumor_skull_path)):
        if tumor_patients[j] == tumor_skull_path[i].split('/')[-2]:
            tumor_skull_path_order.append(tumor_skull_path[i])
    for i in range(len(tumor_brain_masks)):
        if tumor_patients[j] == tumor_brain_masks[i].split('/')[-2]:
            tumor_brain_masks_order.append(tumor_brain_masks[i])
tumor_skull_path = tumor_skull_path_order
tumor_brain_masks = tumor_brain_masks_order  
print("Num of Healthy: {}, Tumor: {}" .format(len(skull_brain_path),len(tumor_skull_path)))  

In [ ]:
n_pc = 4096
n_patch = 64
c_359_unet = np.load('data_utils/center_c359_UnetSeg.npy')
c_t125_unet = np.load('data_utils/center_t125_UnetSeg.npy')
c_NFBS_unet = np.load('data_utils/center_NFBS_UnetSeg.npy')
c_LBPA_unet = np.load ('data_utils/center_LBPA_UnetSeg.npy')

re_sample = True
re_template = False
if re_sample ==True:
    n_pc = 4096
    n_patch = int(n_pc**0.5)
    GI_T = []

    for i,mask in enumerate(tqdm.tqdm(tumor_brain_masks)):

        input_v = process_scan(mask,mask=True,output_shape=(192,192,192),center=center_T[i])
        smooth = False
        if smooth == False:
            vertices, triangles = mcubes.marching_cubes(input_v, 0)
        else:
            smoothed_sphere = mcubes.smooth(input_v)

        mcubes.export_obj(vertices, triangles, 'Tcurrent.obj')
        m = trimesh.load_mesh('Tcurrent.obj')
        processed_data_single_patient = process_point_cloud_single_patient(m.vertices,c_t125_unet[i],n_patch,n_patch,catesian=True)
        if processed_data_single_patient.shape[0]< n_patch*n_patch:
            print(i,processed_data_single_patient.shape[0])
        GI_T.append(processed_data_single_patient)

    GI_H = []

    for i,mask in enumerate(tqdm.tqdm(brain_masks)):

        input_v = process_scan(mask,mask=True,output_shape=(192,192,192),center=center_H[i])
        smooth = False
        if smooth == False:
            vertices, triangles = mcubes.marching_cubes(input_v, 0)
        else:
            smoothed_sphere = mcubes.smooth(input_v)

        mcubes.export_obj(vertices, triangles, 'Hcurrent.obj')
        m = trimesh.load_mesh('Hcurrent.obj')
        processed_data_single_patient = process_point_cloud_single_patient(m.vertices,c_359_unet[i],n_patch,n_patch,catesian=True)
        if processed_data_single_patient.shape[0]< n_patch*n_patch:
            print(i,processed_data_single_patient.shape[0])
        GI_H.append(processed_data_single_patient)

        
    np.save('data_utils/cGI_T_{}rpt_preC.npy'.format(n_pc),np.array(GI_T))
    np.save('data_utils/cGI_H_{}rpt_preC.npy'.format(n_pc),np.array(GI_H))

    if re_template == True:
        center_tem  = np.median(np.argwhere(mask_template_img_ants.numpy()>0.5),axis=0).astype(int)
        input_v = process_scan(mask_template_img_path,mask=True,output_shape=(192,192,192),center= center_tem)
        smooth = False
        if smooth == False:
            vertices, triangles = mcubes.marching_cubes(input_v, 0)
        else:
            smoothed_sphere = mcubes.smooth(input_v)

        mcubes.export_obj(vertices, triangles, 'Hcurrent.obj')
        m = trimesh.load_mesh('Hcurrent.obj')
        processed_data_single_patient = process_point_cloud_single_patient(m.vertices,m.centroid,n_patch,n_patch,catesian=True)
        if processed_data_single_patient.shape[0]< n_patch*n_patch:
            print(i,processed_data_single_patient.shape[0])
        np.save('data_utils/cGI_tem_{}rpt_preC.npy'.format(n_pc),processed_data_single_patient) 
else:
    GI_T =  np.load('data_utils/cGI_T_{}rpt_preC.npy'.format(n_pc))
    GI_H = np.load('data_utils/cGI_H_{}rpt_preC.npy'.format(n_pc))
    GI_tem = np.load('data/_utilscGI_tem_{}rpt.npy'.format(n_pc))

In [ ]:
def load_H_extra(base_path):
    skull_paths_LBPA40 = []
    brain_mask_paths_LBPA40 = []

    folder  = "LBPA40"
    # , ""]:
    folder_path = os.path.join(base_path, folder)

    # Iterate over each patient folder in the subfolder
    for patient_folder in os.listdir(folder_path):
        patient_path = os.path.join(folder_path, patient_folder)

        # Check if it's a directory
        if os.path.isdir(patient_path):
            skull_file = os.path.join(patient_path, "T1.nii.gz")
            brain_mask_file = os.path.join(patient_path, "mask.nii.gz")

            # Check if the skull and brain mask files exist
            if os.path.isfile(skull_file):
                skull_paths_LBPA40.append(skull_file)
            if os.path.isfile(brain_mask_file):
                brain_mask_paths_LBPA40.append(brain_mask_file)

    skull_paths_NFBS= []
    brain_mask_paths_NFBS = []
    folder  = "NFBS"

    folder_path = os.path.join(base_path, folder)

    # Iterate over each patient folder in the subfolder
    for patient_folder in os.listdir(folder_path):
        patient_path = os.path.join(folder_path, patient_folder)

        # Check if it's a directory
        if os.path.isdir(patient_path):
            skull_file = os.path.join(patient_path, "T1.nii.gz")
            brain_mask_file = os.path.join(patient_path, "mask.nii.gz")

            # Check if the skull and brain mask files exist
            if os.path.isfile(skull_file):
                skull_paths_NFBS.append(skull_file)
            if os.path.isfile(brain_mask_file):
                brain_mask_paths_NFBS.append(brain_mask_file)

    print('Extra Dataset LBPA40--patients :{}   NFBS--patients {}'.format(len(skull_paths_LBPA40),len(skull_paths_NFBS)))

    return skull_paths_LBPA40, brain_mask_paths_LBPA40, skull_paths_NFBS, brain_mask_paths_NFBS

base_path = 'Health_extra' 
skull_paths_LBPA40, brain_mask_paths_LBPA40, skull_paths_NFBS, brain_mask_paths_NFBS =load_H_extra(base_path)